In [ ]:
import os

# Set your directory
os.chdir(r"C:\Users\aldoh\OneDrive\Escritorio\Proyecto LLM Portx2")
print(f"Current directory: {os.getcwd()}")

In [ ]:

#!python preprocess_metrics.py
#!python preprocess_sdm.py
#!python preprocess_publications.py

In [ ]:
# Installed Dependencies
#%pip install crewai
#%pip install python-dotenv
#%pip install anthropic


import os
from dotenv import load_dotenv
import anthropic
from crewai import Agent, Task, Crew, Process, LLM
import re
from collections import defaultdict
import json
from typing import List, Dict, Any 

load_dotenv()

# ============================================================================
# LLM CONFIGURATIONS -------- Anthropic and OpenAI
# ============================================================================
llm = LLM(
    model="claude-sonnet-4-5-20250929",
    api_key=os.getenv("ANTHROPIC_API_KEY"),
)
llmc = LLM(
    model="claude-3-5-haiku-20241022",    # Cheaper
    api_key=os.getenv("ANTHROPIC_API_KEY"),
)
###################################################################
llmgptc = LLM(
    model="gpt-4o-mini-2024-07-18",   # Cheaper
    api_key=os.getenv("OPENAI_API_KEY"),
    response_format={"type": "json_object"},             
)
llmgpt = LLM(
    model="gpt-4o-2024-08-06",        
    api_key=os.getenv("OPENAI_API_KEY"),
)

# ============================================================================
# IMPORT SCHEMAS
# ============================================================================
from schemas import (
    ValidationReport,
    InsightsReport, 
    FactCheckReport,
    CampusInsight,
    CampusValidation,
    CampusFactCheck
)

# ============================================================================
#Process Publications Script
#=============================================================================
#!python preprocess_publications.py
#                                   *Why run this script?*
# This script preprocesses and analyzes social media publication data.
# It filters top publications by creating a engagement score.
# This is needed so we don't have memory issues with the LLM and CrewAI stored data. 
#=============================================================================

# ============================================================================
# AGENTS DEFINITION
# ============================================================================


agent_validator = Agent(
    role="Data Completeness Validator",

    goal="Validate that ALL 20 campuses have complete data across three JSON files. "
         "CRITICAL: You must check EXACTLY 20 campuses - no more, no less. "
         "Use the EXACT campus_id from the JSON files - do NOT invent new IDs. "
         "Generate a ValidationReport with CampusValidation for each of the 20 campuses.",

    backstory="You are a meticulous data validator who NEVER stops until all 20 campuses are validated. "
              "You load all three JSON files and extract the EXACT campus_id values from them. "
              "You create one CampusValidation entry per campus using their actual campus_id from the data. "
              "You verify field presence and count exactly how many campuses exist. "
              "CRITICAL RULES: "
              "(1) Always validate all 20 campuses - never stop early "
              "(2) Use exact campus_id from the JSON files - never modify them "
              "(3) Double-check your output contains exactly 20 CampusValidation objects before returning "
              "(4) If you find fewer than 20 campuses in a file, note this in the summary",
    llm=llm,
    verbose=True,
)

agent_insight = Agent(
    role="University Campus Insight Writer",

    goal="Generate EXACTLY 20 comprehensive insights (one per campus) using only the ACTUAL data from three JSON files. "
         "CRITICAL: You must generate all 20 insights in a single response - do not stop early. "
         "Use ONLY the categoría words (excepcional/sobresaliente/satisfactorio/regular/deficiente) - NEVER show numeric scores. "
         "Calculate percentages from actual current_month vs previous_year_month data.",

    backstory="You are an expert analyst who ALWAYS completes the full task. "
              "Your process: "
              "(1) Load all three JSON files "
              "(2) Create a list to store 20 insights "
              "(3) For EACH campus_id in the data: "
              "    a. Match data across all three files by campus_id "
              "    b. Calculate REAL percentages: ((current - previous) / previous) * 100 "
              "    c. Extract actual publication content from OUTBOUND_POST "
              "    d. Use ONLY categoría words from totales (visibilidad_categoria, resonancia_categoria, salud_de_marca_categoria) "
              "    e. Write insight in Spanish using this EXACT format: "
              "\n\n"
              "Campus [NOMBRE COMPLETO]\n\n"
              "En agosto 2025, el campus [nombre] presentó un desempeño [salud_de_marca_categoria]. "
              "La visibilidad fue [visibilidad_categoria] y la resonancia [resonancia_categoria]. "
              "El alcance [aumentó/disminuyó] un [X.X]% y las interacciones [aumentaron/disminuyeron] un [X.X]% respecto al mismo mes del año anterior. "
              "\n\n"
              "Entre las publicaciones destacadas se encuentran: [mencionar 2-3 temas/contenidos reales de OUTBOUND_POST]. "
              "\n\n"
              "Los comentarios [aumentaron/disminuyeron] un [X.X]% con respecto al año anterior."
              "\n\n"
              "(4) After generating all 20 insights, verify your output contains exactly 20 CampusInsight objects "
              "(5) If you have fewer than 20, continue generating until you reach 20 "
              "\n\n"
              "CRITICAL RULES: "
              "- NEVER show numeric scores (83, 175, etc.) - ONLY use categoría words "
              "- NEVER stop until you have 20 insights "
              "- ALWAYS calculate percentages from actual numbers in the JSON "
              "- NEVER use placeholder data or invented percentages",
    llm=llm,
    verbose=True,
)

agent_quality = Agent(
    role="Insight Accuracy Validator",

    goal="Verify EVERY claim in all 20 campus insights against source JSON files. "
         "For EACH error found, create a FactCheckIssue object with all required fields. "
         "CRITICAL: If you find errors, you MUST populate the issues_found array - never leave it empty.",


    backstory="You are a ruthless fact-checker who catches every inaccuracy. "
              "Your process: "
              "(1) Load all three JSON files "
              "(2) For each of the 20 campus insights: "
              "    a. Extract every numeric claim (percentages, scores, engagement numbers) "
              "    b. Look up actual values in source JSONs by campus_id "
              "    c. Verify percentage calculations: ((current - previous) / previous) * 100 "
              "    d. For EACH error found, create FactCheckIssue with: "
              "       - campus_id "
              "       - campus_name "
              "       - issue_type (percentage_error / score_error / publication_mismatch / invented_data) "
              "       - incorrect_statement (exact quote from insight_text) "
              "       - correct_information (what it should be with calculation) "
              "       - severity (critical/high/medium/low) "
              "    e. Check if numeric scores appear in text (should only show categorías) "
              "    f. Verify categoría words match actual categoria from totales "
              "(3) Count total issues across all campuses "
              "(4) Calculate accuracy rate "
              "\n\n"
              "CRITICAL RULES: "
              "- If insight shows '76.90 puntos' next to 'sobresaliente', flag as error (shouldn't show numbers) "
              "- If percentage is off by >3%, flag with severity='high' "
              "- If publication doesn't exist in top_8_posts, flag with severity='critical' "
              "- If categoría doesn't match actual data, flag with severity='medium' "
              "- ALWAYS populate issues_found array when errors exist - never return empty array if you found problems",


    llm=llm,
    verbose=True,
)

# ============================================================================
# TASKS DEFINITION
# ============================================================================


task_validator = Task(
    description=(
        "Validate data completeness for ALL 20 campuses. "
        "\n\n"
        "STEP-BY-STEP PROCESS: "
        "1. Load Publicaciones_estructuradas_Top8.json and extract ALL campus_id values "
        "2. Load metrics_estructurado.json and extract ALL campus_id values "
        "3. Load sdm_estructurado.json and extract ALL campus_id values "
        "4. Create a master list of unique campus_ids across all files "
        "5. For EACH campus_id in the master list: "
        "   - Check if it exists in publications with top_8_posts "
        "   - Check if it exists in metrics with current_month and previous_year_month "
        "   - Check if it exists in scores with facebook/twitter/instagram/totales "
        "   - Create CampusValidation object using the EXACT campus_id from the files "
        "6. Verify you have exactly 20 CampusValidation objects "
        "7. If fewer than 20, identify which campuses are missing from which files "
        "\n\n"
        "CRITICAL: Do NOT modify campus_ids (CVA not CCV, GDL not CGD, TOL not CTO, QRO not CQR). "
        "CRITICAL: Generate validations for ALL 20 campuses before returning."
    ),

    expected_output="ValidationReport with exactly 20 CampusValidation objects using correct campus_ids from source files",
    output_pydantic=ValidationReport,
    agent=agent_validator,
)

task_insight = Task(
    description=(
        "Generate insights for ALL 20 campuses in ONE response. "
        "\n\n"
        "STEP-BY-STEP PROCESS: "
        "1. Load all three JSON files "
        "2. Extract list of all campus_ids "
        "3. Create empty insights list "
        "4. FOR EACH campus_id (loop through all 20): "
        "   a. Get publications data: top_8_posts for this campus_id "
        "   b. Get metrics data: current_month and previous_year_month for this campus_id "
        "   c. Get scores data: totales for this campus_id "
        "   d. Calculate percentages: "
        "      - alcance_pct = ((current.ALCANCE_TOTAL - prev.ALCANCE_TOTAL) / prev.ALCANCE_TOTAL) * 100 "
        "      - interacciones_pct = ((current.INTERACCIONES_TOTALES - prev.INTERACCIONES_TOTALES) / prev.INTERACCIONES_TOTALES) * 100 "
        "      - comentarios_pct = ((current.POST_COMMENTS__SUM - prev.POST_COMMENTS__SUM) / prev.POST_COMMENTS__SUM) * 100 "
        "   e. Extract publication themes from OUTBOUND_POST (2-3 posts) "
        "   f. Get categorías: totales.visibilidad_categoria, totales.resonancia_categoria, totales.salud_de_marca_categoria "
        "   g. Write insight using ONLY categoría words (NO NUMBERS) "
        "   h. Add CampusInsight to list "
        "5. Verify insights list has 20 items "
        "6. Return InsightsReport with all 20 insights "
        "\n\n"
        "FORMAT RULES: "
        "- Use 'visibilidad fue excepcional' NOT 'visibilidad fue excepcional (175 puntos)' "
        "- Show percentages as 'aumentó un 21.3%' NOT 'aumentó aproximadamente 21%' "
        "- Mention publication themes NOT full OUTBOUND_POST text "
        "\n\n"
        "CRITICAL: Generate all 20 insights in a single response. Do not stop after 1 or 5 or 10. Count to 20."
    ),
    expected_output="InsightsReport with exactly 20 CampusInsight objects, each with calculated percentages and categoría words only",
    output_pydantic=InsightsReport,
    agent=agent_insight,
    context=[task_validator],
)

task_quality = Task(
    description=(
        "Fact-check ALL 20 campus insights against source data. "
        "\n\n"
        "STEP-BY-STEP PROCESS: "
        "1. Load all three JSON files "
        "2. Load insights from previous task "
        "3. Create empty campus_checks list "
        "4. FOR EACH of the 20 insights: "
        "   a. Extract campus_id from insight "
        "   b. Get actual data from JSONs for this campus_id "
        "   c. Check for these error types: "
        "      - NUMERIC SCORES SHOWN: If text contains '(XX puntos)' after categoría → ERROR "
        "      - WRONG PERCENTAGE: Recalculate, if off by >3% → ERROR "
        "      - WRONG CATEGORÍA: Compare to actual totales.X_categoria → ERROR "
        "      - INVENTED PUBLICATION: Check if theme exists in top_8_posts → ERROR "
        "   d. For EACH error: "
        "      - Create FactCheckIssue object "
        "      - Fill ALL fields (campus_id, campus_name, issue_type, incorrect_statement, correct_information, severity) "
        "      - Add to issues_found array "
        "   e. Count verified_claims and total_claims "
        "   f. Set is_accurate = true if no errors, false if errors exist "
        "   g. Add CampusFactCheck to campus_checks list "
        "5. Calculate overall statistics "
        "6. Return FactCheckReport "
        "\n\n"
        "CRITICAL: If you find errors, POPULATE issues_found array. Do not return empty array when errors exist. "
        "CRITICAL: Check all 20 campus insights before returning."
    ),

    expected_output="FactCheckReport with campus_checks for all 20 campuses, with issues_found populated when errors exist",

    output_pydantic=FactCheckReport,
    agent=agent_quality,
    context=[task_insight],
)

# ============================================================================
# DATA LOADING - STRUCTURED JSON FILES (Already Processed by Python Scripts)
# ============================================================================

print("📂 Loading structured JSON files...")
print("=" * 80)

# Load Publications (Standard JSON format - NOT JSON Lines)
print("📂 Loading structured JSON files...")
print("=" * 80)

# Load Publications (JSON LINES format - one object per line)
publicaciones_data = []
with open('Publicaciones_estructuradas_Top8.json', 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            publicaciones_data.append(json.loads(line))

print(f"✅ Publications loaded: {len(publicaciones_data)} campuses")

# Load Metrics (Standard JSON format)
with open('metrics_estructurado.json', 'r', encoding='utf-8') as f:
    metrics_data = json.load(f)

print(f"✅ Metrics loaded: {len(metrics_data['regions'])} regions")

# Load Scores (Standard JSON format)
with open('sdm_estructurado.json', 'r', encoding='utf-8') as f:
    scores_data = json.load(f)

print(f"✅ Scores loaded: {len(scores_data['campuses'])} campuses")

print("\n🎯 All structured JSON files ready for validation")
print("=" * 80)

# ============================================================================
# CREW CREATION - NEW 3-AGENT ARCHITECTURE
# ============================================================================

crew = Crew(
    agents=[
        agent_validator,
        agent_insight,
        agent_quality
    ],
    tasks=[
        task_validator,
        task_insight,
        task_quality
    ],
    process=Process.sequential,
    verbose=True
)

print("\n✅ Crew configured with 3 agents and 3 tasks")
print("   Agent 1: Data Validator")
print("   Agent 2: Insight Generator")
print("   Agent 3: Quality Assurance")
print("=" * 80)

# ============================================================================
# CREW EXECUTION
# ============================================================================

try:
    print("\n🚀 Starting crew execution...")
    print("=" * 80)
    
    # Kick off the crew with file paths (agents will load the files)
    result = crew.kickoff(inputs={
        'publications_file': 'Publicaciones_estructuradas_Top8.json',
        'metrics_file': 'metrics_estructurado.json',
        'scores_file': 'sdm_estructurado.json',
        'month': 'agosto',
        'year': 2025
    })

    # Display intermediate results
    print("\n" + "=" * 80)
    print("📊 INTERMEDIATE RESULTS:")
    print("=" * 80)

    for i, task in enumerate(crew.tasks, 1):
        agent_name = task.agent.role
        print(f"\n{'='*80}")
        print(f"🔹 Task {i}: {agent_name}")
        print(f"{'='*80}")
        
        if hasattr(task, 'output') and task.output:
            output_preview = str(task.output.raw)[:800]
            print(f"{output_preview}...")
            
            # Parse and show key metrics
            try:
                if i == 1:  # Validator
                    print(f"\n📋 Validation Summary:")
                    print(f"   Total Campuses: {task.output.pydantic.total_campuses}")
                    print(f"   Complete: {task.output.pydantic.complete_campuses}")
                    print(f"   Incomplete: {task.output.pydantic.incomplete_campuses}")
                
                elif i == 2:  # Insight Generator
                    print(f"\n📝 Insights Summary:")
                    print(f"   Total Insights Generated: {task.output.pydantic.total_insights}")
                    print(f"   Expected: 20 campuses")
                
                elif i == 3:  # Quality Checker
                    print(f"\n✔️  Quality Check Summary:")
                    print(f"   Campuses Checked: {task.output.pydantic.total_campuses_checked}")
                    print(f"   Accurate: {task.output.pydantic.accurate_campuses}")
                    print(f"   With Errors: {task.output.pydantic.campuses_with_errors}")
                    print(f"   Total Issues: {task.output.pydantic.total_issues_found}")
                    print(f"   Accuracy Rate: {task.output.pydantic.overall_accuracy_rate:.1f}%")
                    
            except Exception as e:
                print(f"   (Could not parse structured output: {e})")
        else:
            print("   ⚠️  No output available")
    
    # Display final result
    print("\n" + "=" * 80)
    print("✅ CREW EXECUTION COMPLETED SUCCESSFULLY")
    print("=" * 80)
    
    print("\n📊 FINAL OUTPUT:")
    print("=" * 80)
    print(result)
    
    # Save outputs to files
    print("\n💾 Saving outputs to files...")
    
    try:
        # Save Validation Report
        if hasattr(crew.tasks[0], 'output') and crew.tasks[0].output:
            with open('validation_report.json', 'w', encoding='utf-8') as f:
                json.dump(crew.tasks[0].output.pydantic.model_dump(), f, ensure_ascii=False, indent=2)
            print("   ✅ Validation report saved: validation_report.json")
        
        # Save Insights Report
        if hasattr(crew.tasks[1], 'output') and crew.tasks[1].output:
            with open('insights_report.json', 'w', encoding='utf-8') as f:
                json.dump(crew.tasks[1].output.pydantic.model_dump(), f, ensure_ascii=False, indent=2)
            print("   ✅ Insights report saved: insights_report.json")
        
        # Save Quality Report
        if hasattr(crew.tasks[2], 'output') and crew.tasks[2].output:
            with open('quality_report.json', 'w', encoding='utf-8') as f:
                json.dump(crew.tasks[2].output.pydantic.model_dump(), f, ensure_ascii=False, indent=2)
            print("   ✅ Quality report saved: quality_report.json")
    
    except Exception as save_error:
        print(f"   ⚠️  Error saving files: {save_error}")
    
    print("\n" + "=" * 80)
    print("🎉 ALL TASKS COMPLETED!")
    print("=" * 80)
    
except Exception as e:
    print("\n" + "=" * 80)
    print(f"❌ ERROR DURING EXECUTION")
    print("=" * 80)
    print(f"\nError message: {str(e)}")
    print("\nFull traceback:")
    import traceback
    traceback.print_exc()
    print("\n" + "=" * 80)